In [3]:
import sys
import os
import torch
import numpy as np
import shutil

from torch.autograd import Variable
from torchvision import transforms
from PIL import Image
from PIL import ImageFile

sys.path.insert(0, '..')
from data import crop_upper_part
from model import SqueezeModelSoftmax

### Setup parameters

In [9]:
# path to a PyTorch state dict
MODEL_PATH = '../models/26_class_final_all_epoch_48-valLoss_0.01565-valF1_0.99899-testF1_0.99701' 
NUM_CLASSES = 26
INPUT_SHAPE = (3, 370, 400) # C x H x W

# path to a non-annotated dataset where all images are in same folder with name: <integer_id>.jpg
DATASET_PATH = '../../data/robust_ml_challenge_testset'

# path to empty folder (it is not mandatory that it exists) - where annotated dataset will be stored
OUTPUT_ANNOTATED_PATH = '../../data/annotated_ds'

OUTPUT_CSV_PATH = '../output.csv'

NUM_THREADS = 8 # number of threads to use - should be same as number of virtual CPU cores
USE_GPU = True # use CUDA related stuff

ImageFile.LOAD_TRUNCATED_IMAGES = True

### Class indices

In [10]:
class_labels = {
 'Albertsons': 0,
 'BJs': 1,
 'CVSPharmacy': 2,
 'Costco': 3,
 'FredMeyer': 4,
 'Frys': 5,
 'HEB': 6,
 'HarrisTeeter': 7,
 'HyVee': 8,
 'JewelOsco': 9,
 'KingSoopers': 10,
 'Kroger': 11,
 'Meijer': 12,
 'Other': 13,
 'Publix': 14,
 'Safeway': 15,
 'SamsClub': 16,
 'ShopRite': 17,
 'Smiths': 18,
 'StopShop': 19,
 'Target': 20,
 'Walgreens': 21,
 'Walmart': 22,
 'Wegmans': 23,
 'WholeFoodsMarket': 24,
 'WinCoFoods': 25}

class_dict = {v: k for k, v in class_labels.items()}

### Helper methods

In [ ]:
def data_preprocess_transformations(input_shape, crop_perc = 0.5):
    """Preprocess object for transforming image to model input

    Args:
        input_shape: model input shape (channels x height x width)
        crop_perc: percent of how much image would be cropped from

    Returns:
        Composite of transforms objects.
        
    """

    num_channels, height, width = input_shape
    
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(num_channels),
        transforms.Resize((height, width)),
        transforms.ToTensor(),
    ])


def predict_image(image, model, preprocess_transformations, use_gpu=False):
    """

    Args:
        image: Input image
        model: Model to predict with
        preprocess_transformations: Image preprocessing transformations

    Returns:
        Model outputs for given image.
        
    """
    
    input_image = preprocess_transformations(image)
    input_image = input_image.unsqueeze(0)
    input_tensor = Variable(input_image)
    
    if use_gpu:
        input_tensor = input_tensor.cuda(0)
    
    return model(input_tensor).cpu().data.numpy()[0]


def list_input_images(images_folder):
    """

    Args:
        images_folder: Folder with input images with name template: <int_id>.jpg

    Returns:
        List of tuples: (image_file_name, image_int_id)
        
    """
    files = os.listdir(images_folder)
    images = []
    
    for file in files:
        
        name_components = file.split(".")
        extension = name_components[1].lower()
        
        if extension == 'jpg' or extension == 'jpeg':
            image_id = int(name_components[0])
            images.append((file, image_id))
            
    return sorted(images, key=lambda x: x[1])


def open_image(image_path):
    """

    Args:
        image_path: Path to an image.

    Returns:
        PIL Image in RGB format.
        
    """
    with open(image_path, 'rb') as f:
        image = Image.open(f).convert("RGB")
    return image


def predicted_store(prediction):
    """

    Args:
        prediction: Model probability output.
        
    Returns:
        Most probable store (argmax)
        
    """
    class_indice = np.argmax(prediction)
    return class_dict[class_indice]


def print_predicted_store_stats(predicted_stores):
    unique, counts = np.unique(predicted_stores, return_counts=True)
    stats_dict = dict(zip(unique, counts))
    for store, count in stats_dict.items():
        print("{:<16} => {}".format(store, count))

### Load model and make predictions (softmax)

In [ ]:
model = SqueezeModelSoftmax(num_classes=NUM_CLASSES)
model_state_dict = torch.load(MODEL_PATH, map_location=lambda storage, loc: storage)
model.load_state_dict(model_state_dict)
model.eval()
torch.set_num_threads(NUM_THREADS)

if USE_GPU:
    model.cuda(0)

In [ ]:
predictions = []
images = list_input_images(DATASET_PATH)
preprocess_transformations = data_preprocess_transformations(INPUT_SHAPE)
num_images = len(images)
for index, (image_filename, _) in enumerate(images):
    
    image_path = os.path.join(DATASET_PATH, image_filename)
    image = open_image(image_path)
    
    prediction = predict_image(image, model, preprocess_transformations, USE_GPU)
    predictions.append(prediction)
    
    print('Image {}/{}'.format(index + 1, num_images), end="\r", flush=True)

### Process predictions

In [ ]:
# TODO: Heuristics 

def some_heuristic_take_all_outputs(predictions):
    return predictions

predictions = some_heuristic_take_all_outputs(predictions)

In [6]:
# Lets make predictions

predicted_stores = [predicted_store(prediction) for prediction in predictions]

### Save and annotate outputs

In [7]:
# Save predicted stores to CSV

with open(OUTPUT_CSV_PATH, "w") as f:
    for i, store in enumerate(predicted_stores):
        f.write(store)
        if i < len(predicted_stores) - 1:
            f.write("\n")

In [ ]:
# Print current statistics

print_predicted_store_stats(predicted_stores)

In [ ]:
# Annotate - copy images to new folder - separated by class

if not os.path.exists(OUTPUT_ANNOTATED_PATH):
    os.makedirs(OUTPUT_ANNOTATED_PATH)
    
num_stores = len(predicted_stores)
    
for index, store in enumerate(predicted_stores):
    target_folder_path = os.path.join(OUTPUT_ANNOTATED_PATH, store)
    if not os.path.exists(target_folder_path):
        os.makedirs(target_folder_path)
    
    filename = str(index) + ".jpg"
    src_file_path = os.path.join(DATASET_PATH, filename)

    shutil.copy2(src_file_path, target_folder_path)
    print('Image {}/{}'.format(index + 1, num_stores), end="\r", flush=True)